<a href="https://colab.research.google.com/github/dmourlot89/MELI_challenge/blob/main/MELI_DSChallenge_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3. Similitud entre productos

**Descripción:**

Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “`items_titles.csv`” que tiene títulos de 30 mil productos de 3 categorías diferentes de Mercado Libre Brasil.

El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares son dos títulos del dataset `item_titles_test.csv` generando como output un listado en el cual, ordenando por score de similitud, podamos encontrar los pares de productos más similares en nuestro dataset de test.

In [2]:
# import required libraries and functions
import numpy as np                     # numerical library 
import pandas as pd                    # Use of daframe structure and operations
from nltk.metrics.distance import *    # Distances from NLTK library
from itertools import product          # Computes cartesian product of input iterables.

### Importar dataset

Cargar titulos contenidos en el archivo `item_titles.csv`

In [3]:
# load dataset
url = 'https://raw.githubusercontent.com/dmourlot89/MELI_challenge/main/items_titles.csv'

titles_df = pd.read_csv(url)

In [7]:
print(f"Shape of titles_df:\n{titles_df.shape}")
titles_df.head()

Shape of titles_df:
(30000, 1)


,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...


La libreria de precesamiento del lenguaje natural NLTK nos ofrece varias opciones para el calculo de distancias entre dos cadenas de texto o dos oraciones. Probemos un par de estas, usando el ejemplo dado en la descripcion del challenge.

In [10]:
title1= 'Zapatillas Nike'
title2= 'Zapatillas Adidas'

# transform a bit for use of NLTK distance functions
label1 = set(title1.lower().split())
label2 = set(title2.lower().split())

print(f"'{title1}' vs '{title2}'\n")
# NLTK's distance functions (use 1 - distance, since we're looking for similarity)
print(f"Jaccard Index (NLTK): {1 - jaccard_distance(label1,label2):.2f}")   # Distance metric comparing set-similarity
print(f"MASI Distance (NLTK): {1 - masi_distance(label1,label2):.2f}")             # Distance metric that takes into account partial agreement

'Zapatillas Nike' vs 'Zapatillas Adidas'

Jaccard Index (NLTK): 0.33
MASI Distance (NLTK): 0.11


Ambas metricas nos dan una estimado valido de cuan similares son dos titulos, aunque no con los resultados que aparecen en el ejemplo de la orden del desafio. Podemos obtener los mismos valores de score para los titulos "Zapatillas Nike" y "Zapatillas Adidas", usando la siguiente funcion con algebra de conjuntos.

In [21]:
def computeSetScore(title1, title2):
  '''
  Computes a similarity score between two given titles using sets algebra
  '''
  label1 = set(title1.lower().split())
  label2 = set(title2.lower().split())

  union = label1.union(label2)
  intersection = label1.intersection(label2)
  complement = union - intersection
  custom_score = 1 if len(complement) == 0 else len(intersection)/len(complement)

  return custom_score

In [26]:
print(f"'{title1}' vs '{title2}'")
print(f"Sets score: {computeSetScore(title1, title2):.2f}\n")
print(f"'{title1}' vs '{title1}'")
print(f"Sets score: {computeSetScore(title1, title1):.2f}")

'Zapatillas Nike' vs 'Zapatillas Adidas'
Sets score: 0.50

'Zapatillas Nike' vs 'Zapatillas Nike'
Sets score: 1.00


### Listado de similitud

Ahora podemos proceder a 

In [84]:
def buildSimilarityList(titles1, titles2):
  '''
  Build a new dataframe using the cartesian product of
  titles1 and titles2 (find all posible combinations of 
  titles in both); then compute and add a similarity score column.

  titles1: One-column dataframe with a set of titles
  titles2: One-column dataframe with a set of titles

  sim_df: Three-columns dataframe with pairs of titles and its
          corresponding similarity score
  '''

  sim_df = pd.merge(titles1.assign(key=1), titles2.assign(key=1), 
                 on='key').drop('key', axis=1)

  sim_df['SIMILARITY_SCORE(0,1)'] = sim_df.apply(lambda row : computeSetScore(row[0],
                                                                              row[1]),
                                                 axis = 1)  
  return sim_df

In [88]:
sim_df = buildSimilarityList(titles_df.iloc[:100,:], titles_df)
sim_df.head()

,ITE_ITEM_TITLE_x,ITE_ITEM_TITLE_y,"SIMILARITY_SCORE(0,1)"
0,Tênis Ascension Posh Masculino - Preto E Verme...,Tênis Ascension Posh Masculino - Preto E Verme...,1.000000
1,Tênis Ascension Posh Masculino - Preto E Verme...,Tenis Para Caminhada Super Levinho Spider Corr...,0.000000
2,Tênis Ascension Posh Masculino - Preto E Verme...,Tênis Feminino Le Parc Hocks Black/ice Origina...,0.066667
3,Tênis Ascension Posh Masculino - Preto E Verme...,Tênis Olympikus Esportivo Academia Nova Tendên...,0.076923
4,Tênis Ascension Posh Masculino - Preto E Verme...,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,0.000000


### Consideraciones sobre el score de similitud

Nuestro score parece funcionar bien para los casos de ejemplo proporcionados. Sin embargo, probemos otros casos posibles.

In [91]:
# Same title only with a couple of changes or mispellings
title1 = 'Zapatillas Nike'
title3= 'Zapatilla Nikes'

label1 = set(title1.lower().split())
label3 = set(title3.lower().split())

print(f"'{title1}' vs '{title3}'\n")
print(f"Sets score: {computeSetScore(title1,title3):.2f}")
print(f"Jaccard Index (NLTK): {1 - jaccard_distance(label1,label3):.2f}")
print(f"MASI Distance (NLTK): {1 - masi_distance(label1,label3):.2f}") 

'Zapatillas Nike' vs 'Zapatilla Nikes'

Sets score: 0.00
Jaccard Index (NLTK): 0.00
MASI Distance (NLTK): 0.00


Como puede apreciarse, nuestro score (tambien las distancias Jaccard y MASI) asigna similitud=0 a dos cadenas practicamente iguales!! 

Algo similar ocurre cuando comparamos los dos primeros titulos de nuestro dataset (ver segunda fila de sim_df, arriba). A pesar de tratarse de productos similares (Tenis, en ambos casos), debido a una pequenha diferencia ortografica, los scores basados en algebra de conjuntos le asignan similitud = 0. 

In [93]:
t1 = titles_df.iloc[0,0]
t2 = titles_df.iloc[1,0]

l1 = set(t1.lower().split())
l2 = set(t2.lower().split())

print(f"Title 1:\n'{t1}'")
print(f"Title 2:\n'{t2}'\n")
print(f"Sets score: {computeSetScore(t1,t2):.2f}")
print(f"Jaccard Index (NLTK): {1 - jaccard_distance(l1,l2):.2f}")
print(f"MASI Distance (NLTK): {1 - masi_distance(l1,l2):.2f}")

Title 1:
'Tênis Ascension Posh Masculino - Preto E Vermelho '
Title 2:
'Tenis Para Caminhada Super Levinho Spider Corrida '

Sets score: 0.00
Jaccard Index (NLTK): 0.00
MASI Distance (NLTK): 0.00


Dependiendo de la logica de negocio, el fallo anterior pudiera ser no solo ineficaz, sino incluso problematico. Probemos una metrica alternativa a ver que tal se desempenha en este caso.

In [94]:
def computeEditDistanceScore(title1, title2):
  '''
  Computes a similarity score using the Levenshtein edit-distance
  '''
  s1 = ''.join(title1.lower().split())
  s2 = ''.join(title2.lower().split())

  edit_dist = edit_distance(s1, s2)    # number of actions needed to transform s1 into s2

  # a small computation to transform edit_dist into a 0-1 index
  edit_score = 1/(edit_dist + 0.01)
  
  return edit_dist, edit_score


In [95]:
print(f"'{title1}' vs '{title3}'\n")
s1 = ''.join(title1.split())
s3 = ''.join(title3.split())
numb_edits, edit_score = computeEditDistanceScore(title1,title3)
print(f"Number of edits: {numb_edits}")
print(f"Edit score: {edit_score:.2f}\n")


print(f"'{t1}' vs '{t2}'\n")
s1 = ''.join(t1.lower().split())
s2 = ''.join(t2.lower().split())
numb_edits, edit_score = computeEditDistanceScore(t1,t2)
print(f"Number of edits: {numb_edits}")
print(f"Edit score: {edit_score:.2f}")

'Zapatillas Nike' vs 'Zapatilla Nikes'

Number of edits: 2
Edit score: 0.50

'Tênis Ascension Posh Masculino - Preto E Vermelho ' vs 'Tenis Para Caminhada Super Levinho Spider Corrida '

Number of edits: 33
Edit score: 0.03


El score basado en el Levenshtein edit-distance es mas robusto ante errores ortograficos y otros pequenhos cambios entre titulos. 

Una ultima prueba para nuestros scores...

In [97]:
title4= 'Nike Zapatillas'     # invert the order
label4 = set(title4.lower().split())
s4 = ''.join(title4.split())
edit_dist = edit_distance(s1, s4, transpositions = True)    # number of actions needed to transform s1 into s2
# a small computation to trasnform edit_dist into a 0-1 index
edit_score = 1/(edit_dist + 0.01)
print(f"'{title1}' vs '{title4}'\n")
print(f"Sets score: {computeSetScore(title1,title4):.2f}")
print(f"Jaccard Index (NLTK): {1 - jaccard_distance(label1,label4):.2f}")
print(f"MASI Distance (NLTK): {1 - masi_distance(label1,label4):.2f}") 
print(f"Number of edits needed: {edit_dist}")
print(f"Edit-Distance score: {edit_score:.2f}")


'Zapatillas Nike' vs 'Nike Zapatillas'

Sets score: 1.00
Jaccard Index (NLTK): 1.00
MASI Distance (NLTK): 1.00
Number of edits needed: 36
Edit-Distance score: 0.03


En este ultimo caso nuestro score (al igual que la distancias Jaccard y MASI) reconocen ambos titulos como iguales; esto es, estan compuestos por los mismos elementos (pero sin notar el cambio de orden). Por su parte, para el edit_distance se trata de dos cadenas diferentes que necesitan transformaciones (8 en total). Esto puede resultar de utilidad -aun sin ser optimo. 

**Propuesta:**

Dependiendo de las reglas y la logica del negocio, podria ser mas util calcular un score mas robusto, capaz de otorgar valores sensatos aun cuando existan diferencias entre los titulos (orden de los elementos, ortografia, etc.) En este caso, propongo un promedio de nuestros scores basados en algebra de conjuntos y Levenshtein edit-distance, respectivamente.

In [20]:
def customScore2(title1, title2):
  '''
  Computes two similarity scores, using set algebra
  and Levenshtein edit-distance respectively, then 
  return the average of the two.
  '''

  # convert to lowercase in order to ignore capitalization
  # differences, then create sets
  label1 = set(title1.lower().split())
  label2 = set(title2.lower().split())
  
  union = label1.union(label2)
  intersection = label1.intersection(label2)
  complement = union - intersection
  set_score = 1 if len(complement) == 0 else len(intersection)/len(complement)

  # prepare strings for NLTK's edit_distance() function
  s1 = ''.join(titles_df.iloc[0,0].split())
  s2 = ''.join(titles_df.iloc[2,0].split())
  edit_dist = edit_distance(s1, s2, transpositions = True)    # number of actions needed to transform s1 into s2
  # a small computation to trasnform edit_dist into a 0-1 index
  edit_score = 1/(edit_dist + 0.01)

  return (set_score + edit_score)/2